---
# Section 2.6: Propagation of Roundoff Errors in Gaussian Elimination
---

The $n \times n$ Hilbert matrix $H$ is defined by
$$h_{ij} = \frac{1}{i+j-1}, \quad i,j = 1,\ldots,n.$$
This matrix is known to be very ill-conditioned.

Catastrophic cancellation during Gaussian elimination can occur on such badly conditioned matrices.

In [1]:
n = 7
H = Rational{BigInt}[1//(i+j-1) for i=1:n, j=1:n]

7x7 Array{Rational{BigInt},2}:
 1//1  1//2  1//3  1//4   1//5   1//6   1//7 
 1//2  1//3  1//4  1//5   1//6   1//7   1//8 
 1//3  1//4  1//5  1//6   1//7   1//8   1//9 
 1//4  1//5  1//6  1//7   1//8   1//9   1//10
 1//5  1//6  1//7  1//8   1//9   1//10  1//11
 1//6  1//7  1//8  1//9   1//10  1//11  1//12
 1//7  1//8  1//9  1//10  1//11  1//12  1//13

The inverse of $H$ is integer.

In [5]:
inv(H)

7x7 Array{Rational{BigInt},2}:
     49//1     -1176//1       8820//1  …      -38808//1      12012//1
  -1176//1     37632//1    -317520//1        1596672//1    -504504//1
   8820//1   -317520//1    2857680//1      -15717240//1    5045040//1
 -29400//1   1128960//1  -10584000//1       62092800//1  -20180160//1
  48510//1  -1940400//1   18711000//1     -115259760//1   37837800//1
 -38808//1   1596672//1  -15717240//1  …   100590336//1  -33297264//1
  12012//1   -504504//1    5045040//1      -33297264//1   11099088//1

In [4]:
invH = round(Int, inv(H))

7x7 Array{Int64,2}:
     49     -1176       8820     -29400       48510      -38808      12012
  -1176     37632    -317520    1128960    -1940400     1596672    -504504
   8820   -317520    2857680  -10584000    18711000   -15717240    5045040
 -29400   1128960  -10584000   40320000   -72765000    62092800  -20180160
  48510  -1940400   18711000  -72765000   133402500  -115259760   37837800
 -38808   1596672  -15717240   62092800  -115259760   100590336  -33297264
  12012   -504504    5045040  -20180160    37837800   -33297264   11099088

In [7]:
invH*H

7x7 Array{Rational{BigInt},2}:
 1//1  0//1  0//1  0//1  0//1  0//1  0//1
 0//1  1//1  0//1  0//1  0//1  0//1  0//1
 0//1  0//1  1//1  0//1  0//1  0//1  0//1
 0//1  0//1  0//1  1//1  0//1  0//1  0//1
 0//1  0//1  0//1  0//1  1//1  0//1  0//1
 0//1  0//1  0//1  0//1  0//1  1//1  0//1
 0//1  0//1  0//1  0//1  0//1  0//1  1//1

In [6]:
round(Int, invH*H)

7x7 Array{Int64,2}:
 1  0  0  0  0  0  0
 0  1  0  0  0  0  0
 0  0  1  0  0  0  0
 0  0  0  1  0  0  0
 0  0  0  0  1  0  0
 0  0  0  0  0  1  0
 0  0  0  0  0  0  1

We will first compute the $LU$-decomposition of $H$ using fractional arithmetic to find the _exact_ values of $U$.

In [8]:
L, U, p = lu(H);
U

7x7 Array{Rational{BigInt},2}:
 1//1  1//2   1//3     1//4      1//5        1//6        1//7       
 0//1  1//12  1//12    3//40     1//15       5//84       3//56      
 0//1  0//1   5//504   1//63     2//105    100//4851    25//1176    
 0//1  0//1   0//1    -1//1800  -1//875     -1//616     -1//504     
 0//1  0//1   0//1     0//1      1//40425    5//77616    5//45864   
 0//1  0//1   0//1     0//1      0//1        1//620928   1//203840  
 0//1  0//1   0//1     0//1      0//1        0//1        1//37837800

Now let's store the Hilbert matrix $H$ in double floating-point format.

In [10]:
Hd = map(Float64, H)

7x7 Array{Float64,2}:
 1.0       0.5       0.333333  0.25      0.2        0.166667   0.142857 
 0.5       0.333333  0.25      0.2       0.166667   0.142857   0.125    
 0.333333  0.25      0.2       0.166667  0.142857   0.125      0.111111 
 0.25      0.2       0.166667  0.142857  0.125      0.111111   0.1      
 0.2       0.166667  0.142857  0.125     0.111111   0.1        0.0909091
 0.166667  0.142857  0.125     0.111111  0.1        0.0909091  0.0833333
 0.142857  0.125     0.111111  0.1       0.0909091  0.0833333  0.0769231

The following $LU$-decomposition is computed using floating-point arithmetic.

In [11]:
Ld, Ud, pd = lu(Hd);
Ud

7x7 Array{Float64,2}:
 1.0  0.5        0.333333    0.25        …   0.166667     0.142857   
 0.0  0.0833333  0.0888889   0.0833333       0.0694444    0.0634921  
 0.0  0.0        0.00634921  0.0107143       0.014881     0.0156986  
 0.0  0.0        0.0         0.00104167      0.0031002    0.00381563 
 0.0  0.0        0.0         0.0            -8.5034e-5   -0.000142714
 0.0  0.0        0.0         0.0         …   8.85771e-7   2.6759e-6  
 0.0  0.0        0.0         0.0             0.0         -3.00325e-8 

Comparing the true $U$ with the $U_d$ that was computed using floating point arithmetic, we see that there a complete loss of precision.

In [13]:
map(Float64, U)

7x7 Array{Float64,2}:
 1.0  0.5        0.333333     0.25         …   0.166667     0.142857   
 0.0  0.0833333  0.0833333    0.075            0.0595238    0.0535714  
 0.0  0.0        0.00992063   0.015873         0.0206143    0.0212585  
 0.0  0.0        0.0         -0.000555556     -0.00162338  -0.00198413 
 0.0  0.0        0.0          0.0              6.44197e-5   0.000109018
 0.0  0.0        0.0          0.0          …   1.61049e-6   4.90581e-6 
 0.0  0.0        0.0          0.0              0.0          2.64286e-8 

In [14]:
norm(Ud - map(Float64,U))/norm(map(Float64, U))

0.019546392954072952

However, the error in the $LU$-decomposition of $H$ is small.

In [15]:
E = Ld*Ud - Hd[pd,:]

7x7 Array{Float64,2}:
 0.0  0.0  0.0          0.0   0.0           0.0          0.0        
 0.0  0.0  0.0          0.0   0.0           0.0          0.0        
 0.0  0.0  0.0          0.0   0.0          -1.38778e-17  0.0        
 0.0  0.0  0.0          0.0   0.0           2.77556e-17  0.0        
 0.0  0.0  0.0          0.0   0.0          -1.38778e-17  0.0        
 0.0  0.0  2.77556e-17  0.0   0.0           1.38778e-17  1.38778e-17
 0.0  0.0  0.0          0.0  -1.38778e-17  -1.38778e-17  0.0        

In [16]:
norm(E)/norm(Hd)

2.5463499572255483e-17

# Solving $Hx = b$

In [17]:
n = 16
H = Rational{BigInt}[1//(i+j-1) for i=1:n, j=1:n]
x = ones(Rational{BigInt}, n)
b = H*x
xhat = H\b

16-element Array{Rational{BigInt},1}:
 1//1
 1//1
 1//1
 1//1
 1//1
 1//1
 1//1
 1//1
 1//1
 1//1
 1//1
 1//1
 1//1
 1//1
 1//1
 1//1

In [18]:
n = 16
H = Float64[1/(i+j-1) for i=1:n, j=1:n]
x = ones(n)
b = H*x
xhat = H\b

16-element Array{Float64,1}:
    0.999998
    1.00034 
    0.98695 
    1.21659 
   -0.900469
   10.7057  
  -28.6776  
   52.3557  
  -32.8933  
  -39.1793  
   89.7519  
  -17.2069  
 -102.192   
  131.66    
  -65.6795  
   14.0522  

The computed solution $\hat{x}$ is very far from the true solution $x$.

In [19]:
norm(xhat - x)/norm(x)

54.15518954564602

The Hilbert matrix is very ill-conditioned, so solving $Hx=b$ is very sensitive to roundoff errors. Here we compute the condition number $\kappa_2(H)$.

In [20]:
cond(H)

7.865467778431645e17

However, the residual $\hat{r} = b - H\hat{x}$ is very small, so $\hat{x}$ exactly satisfies the slightly perturbed system
$$H\hat{x} = b + \delta b,$$
where $\delta b = -\hat{r}$.

In [21]:
rhat = b - H*xhat;
norm(rhat)/norm(b)

5.889624988631708e-16

Therefore, the computation of $\hat{x}$ is **backward stable**. The large error in $\hat{x}$ is completely due to the fact that $H$ is very ill-conditioned.